<a href="https://colab.research.google.com/github/wbw20000/big_model/blob/main/RLHF_Reward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #连接到google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#定义路径，并读取.py文件
file_path = "/content/drive/MyDrive/Colab Notebooks/GPT图解学习代码/08_ChatGPT/RLHF_Reward_ChatGPT.py"
with open(file_path, "r") as f:
    print(f.read())

import torch # 导入torch
from transformers import GPT2Tokenizer # 导入GPT2分词器
from transformers import GPT2LMHeadModel # 导入GPT2语言模型

model_name = "gpt2"  # 也可以选择其他模型，如"gpt2-medium"、"gpt2-large"等
tokenizer = GPT2Tokenizer.from_pretrained(model_name) # 加载分词器
device = "cuda" if torch.cuda.is_available() else "cpu" # 判断是否有可用GPU
model = GPT2LMHeadModel.from_pretrained(model_name).to(device) # 将模型加载到设备上（CPU或GPU）
vocab = tokenizer.get_vocab() # 获取词汇表

# 示例RLHF数据集
data = [
    {
        "User": "What is the capital of France?",
        # "AI": "The capital of France is Paris.",
        "AI": "Paris.",
        "score": 5
    },
    {
        "User": "What is the capital of France?",
        "AI": "Rome.",
        "score": 1
    },
    {
        "User": "How to cook pasta?",
        # "AI": "To cook pasta, first boil water and then add pasta.",
        "AI": "first boil water.",
        "score": 4
    },
    {
        "User": "How to cook pasta?",
        # "AI": "First, turn on the microwave and pu

In [ ]:
# 先运行%load "/content/drive/MyDrive/Colab Notebooks/GPT图解学习代码/08_ChatGPT/RLHF_Reward_ChatGPT.py" 就会自动加载.py的代码进来
import torch # 导入torch
from transformers import GPT2Tokenizer # 导入GPT2分词器
from transformers import GPT2LMHeadModel # 导入GPT2语言模型

model_name = "gpt2"  # 也可以选择其他模型，如"gpt2-medium"、"gpt2-large"等
tokenizer = GPT2Tokenizer.from_pretrained(model_name) # 加载分词器
device = "cuda" if torch.cuda.is_available() else "cpu" # 判断是否有可用GPU
model = GPT2LMHeadModel.from_pretrained(model_name).to(device) # 将模型加载到设备上（CPU或GPU）
vocab = tokenizer.get_vocab() # 获取词汇表

# 示例RLHF数据集
data = [
    {
        "User": "What is the capital of France?",
        # "AI": "The capital of France is Paris.",
        "AI": "Paris.",
        "score": 5
    },
    {
        "User": "What is the capital of France?",
        "AI": "Rome.",
        "score": 1
    },
    {
        "User": "How to cook pasta?",
        # "AI": "To cook pasta, first boil water and then add pasta.",
        "AI": "first boil water.",
        "score": 4
    },
    {
        "User": "How to cook pasta?",
        # "AI": "First, turn on the microwave and put the pasta inside.",
        "AI": "microwave.",
        "score": 2
    }
]


from torch.utils.data import Dataset  # 导入Pytorch的Dataset
class RLHFDataset(Dataset):
    def __init__(self, data, tokenizer, vocab):
        self.tokenizer = tokenizer  # 分词器
        self.vocab = vocab  # 词汇表
        self.input_data, self.target_data, self.scores = self.process_data(data)

    def process_data(self, data):
        input_data, target_data, scores = [], [], []
        for conversation in data:
            user_question = conversation["User"]
            model_answer = conversation["AI"]
            score = conversation["score"]

            input_tokens = self.tokenizer(f"{user_question}", return_tensors="pt")["input_ids"].tolist()[0]
            input_tokens = input_tokens + [tokenizer.eos_token_id]
            input_data.append(torch.tensor(input_tokens, dtype=torch.long))

            target_tokens = self.tokenizer(model_answer, return_tensors="pt")["input_ids"].tolist()[0]
            target_tokens = target_tokens + [tokenizer.eos_token_id]
            target_data.append(torch.tensor(target_tokens, dtype=torch.long))

            scores.append(score) # 将每句话的整体打分，分配给每个token,让他们的打分跟整句话的分数一样。

        return input_data, target_data, scores

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx):
        return self.input_data[idx], self.target_data[idx], self.scores[idx]

rlhf_dataset = RLHFDataset(data, tokenizer, vocab) # 创建ChatDataset对象（实例），这个对象是个数据集包括整理后的input_data,target_data,以及scores等数据，传入文件、分词器和词汇表。不是列表但是可以像列表一样使用。
# 打印数据集中前2个数据示例
for i in range(2):
    input_example, target_example, _ = rlhf_dataset[i]
    print(f"Example {i + 1}:")
    print("Input:", tokenizer.decode(input_example))
    print("Target:", tokenizer.decode(target_example))

from torch.utils.data import DataLoader # 导入Dataloader
tokenizer.pad_token = '<pad>' # 为分词器添加pad token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids('<pad>')
# 定义pad_sequence函数，用于将一批序列补齐到相同长度
def pad_sequence(sequences, padding_value=0, length=None):
    # 计算最大序列长度，如果length参数未提供，则使用输入序列中的最大长度
    max_length = max(len(seq) for seq in sequences) if length is None else length
    # 创建一个具有适当形状的全零张量，用于存储补齐后的序列
    result = torch.full((len(sequences), max_length), padding_value, dtype=torch.long)
    # 遍历序列，将每个序列的内容复制到结果张量中
    for i, seq in enumerate(sequences):
        end = len(seq)
        result[i, :end] = seq[:end]
    return result

# 定义collate_fn函数，用于将一个批次的数据整理成适当的形状
def collate_fn(batch):
    # 从批次中分离源序列、目标序列和分数
    sources, targets, scores = zip(*batch)
    # 计算批次中的最大序列长度
    max_length = max(max(len(s) for s in sources), max(len(t) for t in targets))
    # 使用 pad_sequence 函数补齐源序列和目标序列
    sources = pad_sequence(sources, padding_value=tokenizer.pad_token_id, length=max_length)
    targets = pad_sequence(targets, padding_value=tokenizer.pad_token_id, length=max_length)
    # 将分数转换为张量
    scores = torch.tensor(scores, dtype=torch.float)
    # 返回补齐后的源序列、目标序列和分数
    return sources, targets, scores

# 创建Dataloader
batch_size = 2
chat_dataloader = DataLoader(rlhf_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# 检查Dataloader输出
for input_batch, target_batch, score_batch in chat_dataloader:
    print("Input batch tensor size:", input_batch.size())
    print("Target batch tensor size:", target_batch.size())
    print("Score batch tensor size:", score_batch.size())
    break


# 奖励函数
# def reward_function(predictions, targets, scores):
#     correct = (predictions == targets).float() * scores.unsqueeze(1)
#     reward = correct.sum(dim=-1) / (targets != tokenizer.pad_token_id).sum(dim=-1).float()
#     return reward

# def reward_function(predictions, targets, scores):
#     correct = (predictions == targets).float()
#     num_correct = correct.sum(dim=-1)
#     num_total = (targets != tokenizer.pad_token_id).sum(dim=-1).float()
#     match_ratio = num_correct / num_total
#     reward = match_ratio * scores
#     return reward

def reward_function(predictions, targets, scores): #predictions（预测结果），真实 targets（目标序列），以及 scores（人工偏好打分）
    correct = (predictions == targets).float() * scores.unsqueeze(1)
    # predictions == targets：逐 token 比较，得到一个布尔张量（True/False）
    # 举例说明，逐位置对比每个维度的特征
    # predictions = torch.tensor([[1, 2, 3], [4, 5, 6]]) ，这里代表2个批次，及2句话的，并行输入所，并行预测出来的对应的2句话。1，2，3 是整数，代表在vocab词汇表中对应的索引。
    # targets     = torch.tensor([[1, 0, 3], [4, 0, 6]])
    # tensor([[ True, False,  True],
    #     [ True, False,  True]])
    # tensor([[1.0, 0.0, 1.0],
    #    [1.0, 0.0, 1.0]])
    #scores.unsqueeze(1) 的矩阵变换
    # scores = torch.tensor([0.8, 0.6, 0.9])  # shape: [3]
    #tensor([[0.8],
    #    [0.6],） # shape: [2, 1]
    #最终相乘相当于2，3矩阵乘以2，1矩阵，得到2，1矩阵
    #.float()：转为 0.0 / 1.0
    #scores.unsqueeze(1)：从 [batch] 变成 [batch, 1]，广播乘法用
    #最终的相乘结果，“在每个 token 位置上，如果预测正确，就得到这个样本的 score；如果预测错，得分是 0。
    #这是一种 “带权的正确预测掩码”，它实现：
    #功能	描述 正确 token 才有奖励	错的地方直接乘 0，没奖励
    # 分数参与计算	更好样本有更高奖励（如人类评价好）
    # 后续可求总 reward	用 .sum(dim=-1) 计算每个样本的总得分
    reward = correct.sum(dim=-1) / (targets != tokenizer.pad_token_id).sum(dim=-1).float()
    return reward / scores.max()
    # reward = 最终相乘相当于2，3矩阵乘以2，1矩阵，得到2，1矩阵，([[0.8],
    #                                 [0.6],）
    # shape: [2, 1]
    # 假设scores = torch.tensor([0.8, 0.6, 0.4])
    #scores.max() → 0.8
    #reward / scores.max()  → [1.0, 0.75, 0.5]
    #这就是最终 归一化的 reward（奖励），通常是用来指导策略训练的。
import numpy as np
import torch.nn as nn
import torch.optim as optim
# 训练过程
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 100
for epoch in range(num_epochs):
    epoch_rewards = []

    for batch_idx, (input_batch, target_batch, score_batch) in enumerate(chat_dataloader): #编码后的用户问题（标签序列），编码后的 AI 回答（标签序列），	每个 token 的打分（RLHF 得分）
        optimizer.zero_grad()
        input_batch, target_batch = input_batch.to(device), target_batch.to(device)
        score_batch = score_batch.to(device)

        outputs = model(input_batch)
        logits = outputs.logits # logits：每个 token 对应 vocab 中每个词的打分

        _, predicted_tokens = torch.max(logits, dim=-1) # 取每个位置预测概率最大的 token ，#predicted_tokens 是模型输出的序列结果（用于计算 reward）
        # torch.max(logits, dim=-1)返回两个值，values, indices = torch.max(input, dim)，
        #values: 每个位置上 最大值（即最大概率/logit 分数）
        # indices: 每个位置上 最大值的索引，也就是 预测出来的 token ID
        #_,是占位符表示，不关心该位置的数据。

        # 计算奖励
        rewards = reward_function(predicted_tokens, target_batch, score_batch)

        # 计算损失
        loss = criterion(logits.view(-1, logits.size(-1)), target_batch.view(-1)) # criterion 比较 logits（预测分布）和 target（正确词索引）  得到一个交叉熵损失（即预测错误程度）
''' 解释criterion
| `criterion` 是干嘛的？ | 是“判卷老师”，看你预测和答案差多远                    |
| 它怎么判断？            | 把你预测的分数变成概率，然后看你给正确答案的概率是不是高          |
| 概率错得多怎么办？         | 扣你更多分（loss大）                          |
| 概率越准怎么样？          | 扣分越少（loss小）                           |
| 最后模型干嘛？           | 用这些 loss 倒推自己哪里错了，然后慢慢学得更准（这叫反向传播+优化） |
'''
'''对criterion举例说明
想象你在参加一个选择题考试：

每题 4 个选项（A、B、C、D）

你不写 A/B/C/D，而是告诉老师你觉得每个选项的可能性是多少

比如你说：

我觉得 A 有 70% 可能是对的，B 20%，C 5%，D 5%

如果正确答案就是 A，那老师就会说：

你答得挺好，给你扣一点点分（loss 小）

但如果你答成：

我觉得 A 只有 10%，B 40%，C 40%，D 10%

那老师就会说：

你完全错了，扣你很多分（loss 大）
'''
        # 计算加权损失 用 奖励分数（reward） 来决定每个样本的损失要扣多少分：奖励高 → 少扣分，奖励低 → 多扣分。最后把所有人的扣分加起来平均一下，就是 weighted_loss。
        # 1 - rewards rewards
        # 1.0	     0.0	奖励高 → 不惩罚
        # rewards	1 - rewards	意思
        # 0.2	    0.8	    奖励低 → 多惩罚
        #rewards.numel() 是 reward 中元素的个数（即样本数量或 token 数量）
        #torch.sum(...)把所有样本的加权 loss 累加起来
        weighted_loss = torch.sum(loss * (1 - rewards)) / rewards.numel()

        # 反向传播和优化
        weighted_loss.backward()
        # loss.backward()
        optimizer.step()

        epoch_rewards.append(rewards.cpu().numpy())

    avg_reward = np.mean(np.concatenate(epoch_rewards))
    if (epoch + 1) % 20 == 0:
        print(f'Epoch: {epoch + 1:04d}, cost = {weighted_loss:.6f}, avg_reward = {avg_reward:.6f}')



def generate_text_beam_search(model, input_str, max_len=50, beam_width=5):
    model.eval()  # 将模型设置为评估模式（不计算梯度）
    # 对输入字符串进行编码，并将其转换为 PyTorch 张量，然后将其移动到相应的设备上（例如 GPU）
    input_tokens = tokenizer.encode(input_str, return_tensors="pt").to(device)
    # 初始化候选序列列表，包含当前输入序列和其对数概率得分（我们从0开始）
    candidates = [(input_tokens, 0.0)]
    # 禁用梯度计算，以加速预测过程
    with torch.no_grad():
        # 迭代生成最大长度的序列
        for _ in range(max_len):
            new_candidates = []
            # 对于每个候选序列
            for candidate, candidate_score in candidates:
                # 使用模型进行预测
                outputs = model(candidate)
                # 获取输出 logits
                logits = outputs.logits[:, -1, :]
                # 获取对数概率得分的 top-k 值（即 beam_width）及其对应的 token
                scores, next_tokens = torch.topk(logits, beam_width, dim=-1)
                final_results = []
                # 遍历 top-k token 及其对应的得分
                for score, next_token in zip(scores.squeeze(), next_tokens.squeeze()):
                    # 在当前候选序列中添加新的 token
                    new_candidate = torch.cat((candidate, next_token.unsqueeze(0).unsqueeze(0)), dim=-1)
                    # 更新候选序列的得分
                    new_score = candidate_score - score.item()
                    # 如果新的 token 是结束符（eos_token），则将该候选序列添加到最终结果中
                    if next_token.item() == tokenizer.eos_token_id:
                        final_results.append((new_candidate, new_score))
                    # 否则，将新的候选序列添加到新候选序列列表中
                    else:
                        new_candidates.append((new_candidate, new_score))
            # 从新候选序列列表中选择得分最高的 top-k 个序列
            candidates = sorted(new_candidates, key=lambda x: x[1])[:beam_width]
    # 选择得分最高的候选序列
    best_candidate, _ = sorted(candidates, key=lambda x: x[1])[0]
    # 将输出 token 转换回文本字符串
    output_str = tokenizer.decode(best_candidate[0])
    # 移除输入字符串并修复空格问题
    input_len = len(tokenizer.encode(input_str))
    output_str = tokenizer.decode(best_candidate.squeeze()[input_len:])
    return output_str

test_inputs = [
    "What is the capital of France?",
    "How to cook pasta?",
    "hi , what is your name?"
]

for i, input_str in enumerate(test_inputs, start=1):
    generated_text = generate_text_beam_search(model, input_str)
    print(f"Test {i}:")
    print(f"User: {input_str}")
    print(f"AI: {generated_text}")
    print()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 242)

In [ ]:
# 或者通过这条命令直接运行 !python "/content/drive/MyDrive/Colab Notebooks/GPT图解学习代码/08_ChatGPT/RLHF_Reward_ChatGPT.py"